In [1]:
import pandas as pd # 读入csv常用库
# 读取从"https://www.jiqizhixin.com/dailies"爬取的所有2月20-2月27的文章的csv文件
my_data = pd.read_csv("JiQiZhixin.csv")

In [2]:
my_data.head()

,date,title,contents
0,2月20,全国首个人工智能创新发展试验区在北京成立,北京国家新一代人工智能创新发展试验区日前正式成立。这是我国首个国家新一代人工智能创新发展试验...
1,2月20,华为云将在新加坡拓展云平台和人工智能能力,华为云将在新加坡拓展云平台和人工智能能力。（彭博）
2,2月20,卵巢癌预后难判断？AI四倍精准预判仅需四项生理指标,2 月 20 日消息，帝国理工学院和墨尔本大学的研究人员开发出了一款新的 AI 系统 TEX...
3,2月20,优必选参与收购四川九牛足球俱乐部，CEO透露将在国内市场IPO ...,2月20日机器之心消息，人工智能与服务机器人企业优必选科技宣布，联合城市足球集团及中国健腾体...
4,2月20,百度战略投资蝴蝶互动，布局AI小游戏新赛道,2 月 20 日消息，百度宣布战略投资 H5 和小游戏领军企业蝴蝶互动，双方将在小游戏的研发...


In [3]:
my_data.shape

(340, 3)

In [4]:
import jieba

# 停用词处理
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords


stopwords = stopwordslist('停用词表.txt')  # 加载停用词的路径  
# cutted_contents存储所有文章分词的结果
cutted_contents = []
for content in my_data["contents"]:
    # 对于当前content 使用jieba.cut()分词处理
    segs = jieba.cut(content)
    # each_contents_cutted存储当前文章分词处理的结果
    each_contents_cutted = []
    for seg in segs:
        final = ''
        # 若当前得到的分词在停用此表中存在，则剔除当前词
        if seg not in stopwords:
            if seg != '\t':  
                final += seg
                final += " "
                #print(seg)
        each_contents_cutted.append(final)
    # 得到最后的不存在停用词的分词的结果cutted_contents
    cutted_contents.append(each_contents_cutted)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wn/12491ttd6rzf652jst7s6hpr0000gn/T/jieba.cache
Loading model cost 0.873 seconds.
Prefix dict has been built succesfully.


In [5]:
my_data["contents_cutted"] = cutted_contents

In [6]:
my_data.head()

,date,title,contents,contents_cutted
0,2月20,全国首个人工智能创新发展试验区在北京成立,北京国家新一代人工智能创新发展试验区日前正式成立。这是我国首个国家新一代人工智能创新发展试验...,"[北京 , 国家 , 新一代 , 人工智能 , 创新 , 发展 , 试验区 , 日前 , 正..."
1,2月20,华为云将在新加坡拓展云平台和人工智能能力,华为云将在新加坡拓展云平台和人工智能能力。（彭博）,"[华为 , 云 , , , 新加坡 , 拓展 , 云 , 平台 , , 人工智能 , 能力 ..."
2,2月20,卵巢癌预后难判断？AI四倍精准预判仅需四项生理指标,2 月 20 日消息，帝国理工学院和墨尔本大学的研究人员开发出了一款新的 AI 系统 TEX...,"[2 , , 月 , , 20 , , 日 , 消息 , ， , 帝国 , 理工..."
3,2月20,优必选参与收购四川九牛足球俱乐部，CEO透露将在国内市场IPO ...,2月20日机器之心消息，人工智能与服务机器人企业优必选科技宣布，联合城市足球集团及中国健腾体...,"[2 , 月 , 20 , 日 , 机器 , 之心 , 消息 , ， , 人工智能 , , ..."
4,2月20,百度战略投资蝴蝶互动，布局AI小游戏新赛道,2 月 20 日消息，百度宣布战略投资 H5 和小游戏领军企业蝴蝶互动，双方将在小游戏的研发...,"[2 , , 月 , , 20 , , 日 , 消息 , ， , 百度 , , ..."


In [7]:
from jieba.analyse import *

# 采用tf-idf / textrank方法对每一篇文章进行主题词提取 at most 5 topic keywords each article

topic_keywords = []
all_article_non_stop_words = []
for content in my_data["contents_cutted"]:
    #print(content)
    each_article_topic_keywords = []
    each_article_non_stop_words = ' '.join(content)
    all_article_non_stop_words.append(each_article_non_stop_words)
    
    #Method 1：使用jieba中tf-idf选取每篇文章前5名related topic keywords， 筛选"ns":地名， ”n":名词， ”vn“:名动词, "v": 动词
    
    #for keyword, weight in jieba.analyse.extract_tags(each_article_non_stop_words, topK=5, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v')):
        #each_article_topic_keywords.append((keyword, weight))
    #topic_keywords.append(each_article_topic_keywords)
    
    #Method 2：使用jieba中textrank选取每篇文章前5名related topic keywords, 筛选"ns":地名， ”n":名词， ”vn“:名动词, "v": 动词
    
    for keyword, weight in jieba.analyse.textrank(each_article_non_stop_words, topK=5, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v')):
        each_article_topic_keywords.append((keyword, weight))
    topic_keywords.append(each_article_topic_keywords)

# store topic_keywords for each article and its weight (score)
my_data["topic_keywords"] = topic_keywords

#print(temp_temp)

In [8]:
my_data.topic_keywords[133]

[('汽车', 1.0),
 ('电动', 0.6590256872426914),
 ('商议', 0.6552033888205308),
 ('合作', 0.6537100640355884),
 ('智能', 0.6534335888794285)]

In [9]:
my_data.to_csv("JiQiZhiXin_with_topic_keywords.csv",index=False ,encoding='utf_8_sig')

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
import numpy as np

# 使用IF-IDF 提取一周内文章的前10名主题词 

# 提取所有cutted_content word
all_cutted_content = []
for content in my_data["contents_cutted"]:
    for word in content:
        all_cutted_content.append(word)
result = ' '.join(all_cutted_content)

result = [result]

In [11]:
# 使用TF-IDF提取热门主题词
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(result))
    
words = vectorizer.get_feature_names()  #所有文本的关键字
weight = tfidf.toarray()[0]


In [12]:
n = 20 # 前二十位 topic keywords
# 排序
top_keyword_for_week = []
loc = np.argsort(weight)
#print(loc)
length = weight.size - 1
for i in range(n):
    top_keyword_for_week.append((words[loc[length-i]], weight[loc[length-i]]))


In [13]:
print(top_keyword_for_week)

[('人工智能', 0.2565057579488209), ('公司', 0.2311223756518022), ('技术', 0.2137547982906841), ('智能', 0.17901964356844793), ('科技', 0.17634770858981438), ('ai', 0.1696678711432305), ('研究', 0.15630819625006276), ('机器人', 0.15497222876074598), ('5g', 0.15497222876074598), ('合作', 0.14962835880347888), ('提供', 0.14962835880347888), ('发展', 0.12157304152782658), ('医疗', 0.12157304152782658), ('系统', 0.1202370740385098), ('数据', 0.11622917157055948), ('全球', 0.11355723659192593), ('平台', 0.10821336663465883), ('研发', 0.10821336663465883), ('学习', 0.10821336663465883), ('服务', 0.10687739914534206)]


In [14]:
key_words = []
scores = []
for key_word, score in top_keyword_for_week:
    #print("key_words: " + key_word)
    #print("score: " + str(score))
    key_words.append(key_word)
    scores.append(score)


In [15]:
# 将一周内的topic_keyword 转化为DataFrame
topic_keyword_for_week = pd.DataFrame({"topic_keywords" : key_words, "scores" : scores})

In [16]:
# 将一周内的topic_keyword 转化为csv
topic_keyword_for_week.to_csv("JiQiZhiXin_with_topic_keywords_for_week_2_20_to_2_27.csv",index=False ,encoding='utf_8_sig')